# Data Exploration 
Now that we have the streamflow data imported as a CSV file, we move on to the next step in the analytical workflow, namely exploring the data. After we read the data in CSV file back into our session, we will repeat the exploratory analyses we did in Unit1 with Excel:

* Create some scatter plots of the entire dataset as well as for the pre- and post-Falls lake years.


* Deriving some new column in our data frame from existing columns:
 * Create `year` and `month` columns from our `datetime` data
 * Create a `water_year` column based on year and month values
 * Convert our *cfs* data into *cms* and *mgd* units and re-plot


* Summarize the data by *confidence codes*:
 * Group the data by data-value qualification (i.e., "confidence") codes
 * Determine how many different codes exist in our data and what those codes are
 * Compute the number of records in each code
 * Create a quick bar pot showing the record count in each category


* Compute summary statistics of the entire set of discharge data.
 * Compute min, max, mean, median, as well as specific quantiles.
 * Plot these values
 * Repeat for select years of data (pre- and post- Falls Lake construction)

## ♦ Import libraries
First, we need to load a few Python packages build for data analyisis projects. We'll grab familiar `pandas`, but also `matplotlib` a widely-used plotting library, and it's companion `seaborn` which facilitates some plotting operations. 

In [ ]:
#Import modules
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#Instruct Jupyter to allow plots in the document itself
%matplotlib inline

## ♦ Read the data into a *Pandas* dataframe.
Read in the CSV created in the previous notebook into a new Pandas dataframe named `df`. We'll also force the data type of the `site_no` field to be a *string*, not a *number*, which Pandas would infer by default...

In [ ]:
#Load the saved csv file, reading the site_no column as a string, not a number.
df = pd.read_csv('GageData.csv', dtype={'site_no':'str'}) 

In [ ]:
#Confirm it looks good by viewing the first 5 records using the "head()" function
df.head()

### ○ Data types
Dataframes are structured so that each column contains values of a constant and defined **data type**. Typical data types in a Pandas dataframe include integers (`int64`), floating point numbers (`float64`), strings (`object`), and date/time objects (`datetime64`).

If we import data into a dataframe, it infers data types from the the values in the input file unless we override the defaults, as we did above with the `site_no` varible. We can view the datatypes in a dataframe via the dataframe's `dtypes` property:

In [ ]:
#Show the data types of each column
df.dtypes

Note that the `datetime` variable has an incorrectly assigned data type. It is listed as a string (`object`), so we need to re-define it to be a `datetime64` object, specifying the date format our data use.

In [ ]:
#Convert datetime to an actual datetime object
df['datetime'] = pd.to_datetime(df['datetime'],format=('%Y-%m-%d'))
df.dtypes

### ○ Indexes and data slices
Dataframe *indexes* provide different ways to select, sort, and organize data **rows**. By setting our `datetime` field as an index, we can then select rows by date ranges or **"slices"**. 

In [ ]:
#Setting the date time as the index allows time slicing
df.set_index('datetime',inplace=True)

In [ ]:
#Create two new dataframe views: One with records before Falls Lake an one after
dfPre = df.loc['1930-01-01':'1979-12-31']
dfPost = df.loc['1984-01-01':'2017-12-31']

--Some commands to explore a data slice: uncomment each and run this code block--

In [ ]:
#dfPost.head()
#dfPost.shape
#dfPost.columns
#dfPost['MeanFlow_cfs'].max()

► See if you can reveal the overall mean discharges for both the pre- and post- dam data slices? How many records in each? 

In [ ]:
#Show the mean flow before the dam was built...
dfPre['MeanFlow_cfs'].mean()

In [ ]:
#Show the mean flow after the dam was built...
dfPost['MeanFlow_cfs'].mean()

## ♦ Deriving new columns 
We'll want to add a few new columns to our dataset to faciliate analysis. These include:
* `MeanFlow_cms` - mean flow, converted from  $ft^3/s$ to $m^3/s$ (1 cfs = 0.028316847 cms)
* `MeanFlow_mgd` - mean flow, converted from $ft^3/s$ to $MGal/day$ (1 cfs = 0.6464 mgd)
* `year` - The year of the record, extracted from the datetime field
* `month` - The month of the record, extracted from the datetime field
* `water_year` - The water year of the record which extents from Oct. (of the previous year) thru Sept.

And finally, we'll add a column named `status` which will indicate whether the record was collected before construction of Falls Lake (before 1980) or after its completion (after 1984). We could continue to use date slices to compare and plot flow before/after the dam was placed, but creating a new column is more in tune with the "tidy data" concept and will facilitate analysis and plots later... 

#### Creating new flow columns: `MeanFlow_cms` and `MeanFlow_mgd`
* Converting flow units is relatively easy as we just multiply an entire column by the conversion factor...

In [ ]:
#Create a column of flow in cubic meters per second
df['MeanFlow_cms'] = df['MeanFlow_cfs'] * 0.028316847

* <mark>Now you try it with mgd</mark> `(1 cfs = 0.6464  mgd)`

In [ ]:
#Create a column of flow in cubic meters per second
df['MeanFlow_mgd'] = df['MeanFlow_cfs'] * 0.6464

#### Deriving `year` and `month` columns from the `datetime` index
* Creating the year and month columns isn't an arithmetic expression, but rather a data type conversion one. The functions `.year` and `.month` on a datetime object extract the year and month, respectively. 

In [ ]:
#Extract the year from the datetime index
df['year'] = df.index.year

* <mark>Now see if you can create the `month` column...</mark>

In [ ]:
#Extract the month from the datetime index
df['month'] = df.index.month

#### Deriving the `water_year` column from the `year` and `month` data. 
Water year begins in October. Therefore it equals the calendar year for months prior to October, but for October, November, and December, it takes the value of the following year. 

There are two methods for computing water year. Both yield the exact same results but show different data techniques using Pandas. 

##### ♦ <u>Method 1</u> for computing water year: computing values on filtered records
In the first method, we add a new column `water_year` to our data frame, setting its default value to the value in the `year` column. Then we filter for records where the month is >= 10 and revise the value in `water_year` column to be `year + 1`.

In [ ]:
#Create a new column, setting all its values the same as the year column
df['water_year'] = df['year']

In [ ]:
#Create a mask of just records with months from Jan thru Sept (i.e. < 10)
monthMask = df['month'] >= 10

In [ ]:
#Apply this mask to create a view of just records from Jan thru Sept
df.loc[monthMask, 'water_year'] = df['year'] + 1

In [ ]:
#Have a look at a few random records
df.sample(5)

##### ♦ <u>Method 2</u> for computing water year: using a `lambda` function
In this approach, we loop through all records and apply a "lambda" function on each. In this lambda function, the value of "x" is taken from the `datetime` column and the output of the function is passed into the `water_year2` column.

In [ ]:
#Use a lambda function to assign values to water_year2 based on datetime values
df['water_year2'] = df[['year','month']].apply(
    lambda row: row['year'] if row['month'] < 10 else row['year'] + 1, 
    axis = "columns")

In [ ]:
#Show the results; water_year and water_year2 are the same...
df.sample(10)

In [ ]:
#Drop the water_year2 column, now that the demonstration is over...
df.drop("water_year2",axis='columns',inplace=True)

#### Deriving the `status` column with values indicating whether the record was recorded before or after Falls Lake construction.
Lastly, we'll create our status column with two values: `Before FL` for records before 1980, and `After FL` for records after 1984. We do this by calculating new values on slices of rows, using dates to create the slices.

In [ ]:
#Assign records collected before 1980 a status of "Before FL"
df.loc['1930-01-01':'1979-12-31','status'] = "Before FL"
#Assign records collection after 1984 a status of "After FL"
df.loc['1984-01-01':'2017-12-31','status'] = "After FL"
#Show a random sample of the dataframe
df.sample(10)

---
## ♦ Explore the data: Plotting
As we did with Excel, we'd want to explore and examine our data for gaps or outliers. The two approaches for exploring data are to **visualize** the data (e.g. through plots etc.) and to **summarize** our data, both of which might expose odd values or trends in our data.  We'll begin with the former, examining various ways to plot data in Python. Pandas has some plotting capability that will examine. However, we'll also explore the *matplotlib* package, a powerful plotting package that has both Python and R versions, as well as *seaborn* which adds nice aesthetics to our plots. 

#### Create a simple plot using `Pandas` functionality. 
The Pandas library includes some plotting functionality, making it quite easy to generate a limited number of figures from our data. For more information see <br>https://pandas.pydata.org/pandas-docs/stable/visualization.html

* First let's look at the defaul plot, which is a line plot. We specify the value we want to plot, `MeanFlow_cfs`, and without specifying another axis, it uses the index as the independent variable, which in our case is the `datetime` values.

In [ ]:
#Plot using Pandas
df['MeanFlow_cfs'].plot();

In [ ]:
#Repeat above, but add some "aesthetics"
df['MeanFlow_cfs'].plot(
    figsize=(10,5),#figure size
    lw=0.2,        #line weigth
    alpha=0.4,     #transparency
    color='green', #color
    title='Neuse River near Clayton, NC',
);

* Now let's look at **box plots**:https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.boxplot.html. <br>First we'll look at a box plot of all the flow data...

In [ ]:
#Create a box and whiskers plot of our MeanFlow_cms values
df.boxplot('MeanFlow_cms');

 * Now box plots of flow, separated by status 

In [ ]:
#Create a box and whiskers plot of our MeanFlow_cms values, broken by status
df.boxplot('MeanFlow_cms',by='status');

* <mark>Now see if you can produce a box plot of flow data by *month*</mark>

In [ ]:
df.boxplot('MeanFlow_cms',by='month');

* <mark>**CHALLENGE**: See if you can produce the above plot, but just for data after 2010</mark><br>Hint: you might want to involve your index...

In [ ]:
df.loc['2010-12-31':].boxplot('MeanFlow_cms',by='month');

#### Use *ggplot* to plot
An alternative to Pandas built-in plotting capability is [`ggplot`] available through the "[plotnine](https://plotnine.readthedocs.io/en/stable/#)" library. 

In [ ]:
#Import everything from ggplot
from plotnine import *

> If you get an error after running the above, it's possible you haven't installed the plotnine package. To do that, insert a code box and run the following command:
```
!conda install -c anaconda plotnine -y
```
Then re-run the `from plotnine import *` command box above. 

In [ ]:
#Let's bring back our datetime values from index to a column in our dataframe
df['datetime'] = df.index.values

In [ ]:
#Plot using aesthetics
(ggplot(df, aes(x='datetime',y='MeanFlow_cfs'))
       + geom_line()
       + xlab("Year") 
       + ylab("Mean Flow (cfs)")
       + ggtitle("Neuse River near Clayton, NC"))

#### Create a time series plot of mean discharge using `matplotlib`
**Matplotlib** has been described as "the grandfather of plotting" in Python ([source](http://pbpython.com/visualization-tools-1.html)), borrowing much technique (and syntax) from Matlab. It's not the most friendly, but it does a lot and is worth getting the feel for. 

We'll go deeper into visualization tools in the next unit, but we'll examine some examples to get the gist. Meanwhile, more documentation can be found here:<br>
https://matplotlib.org/devdocs/api/pyplot_summary.html

In [ ]:
#Create a plot canvas of a specified size
plt.figure(figsize=(20,6));

#Plot the mean flow (against time)
plt.plot(df['MeanFlow_cfs'],lw=0.3);

#Add a vertical line indicating when Falls Lake construction began
plt.axvline(x='1980-01-01',color='red',ls='--');

#Add some aesthetics
plt.title("Neuse River Near Clayton, NC");
plt.ylabel("Discharge (cfs)");

#### Invoke `Seaborn` to make prettier plots 
https://seaborn.pydata.org/tutorial/aesthetics.html


In [ ]:
#Activate seaborn default aesthetics
sns.set(font_scale=2)
#Repeat above
plt.figure(figsize=(20,6));
plt.plot(df['MeanFlow_cfs'],lw=0.3);
plt.axvline(x='1980-01-01',color='red',ls='--');
plt.title("Neuse River Near Goldsboro, NC");
plt.ylabel("Discharge (cfs)");

### Create a plot of all data, superimposed with the pre- and post-Falls Lake data views


In [ ]:
#Create the canvas
plt.figure(figsize=(20,6))

#Add plot lines for the entire dataframe, then the two dataframe subsets
plt.plot(df['MeanFlow_cfs'],color='grey',linewidth=0.1) 
plt.plot(dfPre['MeanFlow_cfs'],color='green',alpha=0.5,linewidth=0.5)
plt.plot(dfPost['MeanFlow_cfs'],color='blue',alpha=0.5,linewidth=0.5)

#Add the aesthetics
plt.axvline(x='1979-12-31',color='red',ls='--')
plt.title("Neuse River Near Goldsboro, NC")
plt.ylabel("Discharge (cfs)");
plt.annotate('Construction begins',
             xy=('1980',16000),
             xytext=('1955',18000),
             arrowprops=dict(facecolor='red',shrink=0.05))

#Show the plot
plt.show;

### Converting cubic feet/second to cubic *meter*/second

In [ ]:
#Replot (this time in a thinner, orange line)
plt.figure(figsize=(20,6));
plt.plot(df['datetime'],df['MeanFlow_cms'],linewidth=0.5,color='orange')
plt.ylim((0,700))
plt.show;

* <mark> See if you can repeat the plot above, but set the units to `mgd`, make the line `green`, and set the x limit to show just from `1984-01-01` to `2010-01-01`</mark>

In [ ]:
#Replot: modify the commands below..
plt.figure(figsize=(20,6));
plt.plot(df.loc['1984-01-01':'2010-01-01','datetime'],
         df.loc['1984-01-01':'2010-01-01','MeanFlow_mgd'],
         linewidth=0.5,
         color='green')
plt.ylim((0,15000))
plt.show;

## ♦ Explore the data: Summarizing
The alternative to plotting to explore the data is to summarize. Here we'll examine how to examine properties of our dataset as a whole and also how to group our data and examine summaries by those groupings.

### First we'll examine a few basic properties of our dataframe

In [ ]:
#Get a count of all records from the dataframe's shape (rows, columns)
df.shape

In [ ]:
#Or just show the rows, i.e., the first item in the shape result
df.shape[0]

In [ ]:
#We can also use the Python `len` command to show the "lenght" of the dataframe
len(df)

### Summarizing/grouping records by Confidence code
Here, we group the data by the unique values in a column, namely the `Confidence` column. First, we'll just examine the number of unique values and what those values are. 

In [ ]:
#Use "nunique" on the column to list the number of unique values
print(df['Confidence'].nunique())

In [ ]:
#Use "unique" to show what the 4 unique values are
print(df['Confidence'].unique())

Now, we'll **group** the records by confidence codes

In [ ]:
#Create the Pandas GroupBy object
grpConfidence = df.groupby(['Confidence'])

In [ ]:
#We can now list the counts of records by confidence code
grpConfidence.count()

In [ ]:
#Or we can just show the count by a single column
grpConfidence['MeanFlow_cfs'].count()

### Plot counts by confidence code
See: https://pandas.pydata.org/pandas-docs/stable/visualization.html

In [ ]:
#Compute counts and plot them as a bar chart
count_by_Confidence = grpConfidence['MeanFlow_cfs'].count()
count_by_Confidence.plot(kind='pie');

### Generating summary statistics with `Describe`

In [ ]:
#Summarize the MeanFlow_cfs data, using default outputs
df['MeanFlow_cms'].describe()

In [ ]:
#Summarize, **using our own percentiles**
summary_all = df['MeanFlow_cms'].describe(percentiles=[0.1,0.25,0.75,0.9])
summary_all

In [ ]:
#Group by the status values, and then describe..
df.groupby('status')['MeanFlow_cms'].describe(percentiles=[0.1,0.25,0.75,0.9])

In [ ]:
#Group by the status values, and then describe, and then TRANSPOSE for neater presentation
summary_bystatus = df.groupby('status')['MeanFlow_cms'].describe(percentiles=[0.1,0.25,0.75,0.9]).T
summary_bystatus

In [ ]:
#Concatenate (or join) the pre and post summary objects
dfSummary = pd.concat([summary_all, summary_bystatus],axis=1)
dfSummary.columns = ("1930-2017","1930-1980","1984-2017")  #Assign column names to the result
dfSummary

In [ ]:
#Plot bar charts of the three summary tables
dfSummary[4:9].plot(kind='bar',
                    figsize=(20,6),
                    title="Data Summaries: Quantiles");

## Monthly plots
To produce monthly plots, we need to group our data by month, then plot. Grouping by month requires us to extract the month from the datetime column. 

In [ ]:
#Group the rows by month
byMonth = df.groupby('month')

In [ ]:
#Display the mean values of the groups
monthlyDF = byMonth['MeanFlow_cfs'].mean()

In [ ]:
#Reveal the values
monthlyDF

In [ ]:
#Plot the values
monthlyDF.plot(figsize=(20,6),legend=True);

### <mark>♦Exercise♦: Annual plots</mark>
 * Group the `df` dataframe rows by the `year` attribute
 * Compute the total (sum) annual discharge
 * Create a plot of annual discharge by year

In [ ]:
#Group by year
byYear = df.groupby('year')

In [ ]:
#Compute total annual discharge
yearlyDF = byYear['MeanFlow_cfs'].sum()

In [ ]:
#Plot
yearlyDF.plot(figsize=(20,6),legend=True);